## 第九章

- 对象表示形式
    - repr() 以便于开发者理解的方式返回对象的字符串表示形式。
    - str() 以便于用户理解的方式返回对象的字符串表示形式。
    - __bytes__ bytes()函数调用它，获取对象的字节序列表示形式
    - __format__ format()函数和str.format()调用，使用特殊的格式代码显示对象的字符串表示形式
- classmethod与staticmethod方法
    - classmethod
        - 定义操作类，而不是操作实例的方法
        - 类方法的第一个参数是类本身
    - staticmethod
        - 普通方法
        - 第一个参数不是特殊的值
- 格式化显示
    - format(obj, format_spac)
    - str.format(format_spac)
- 私有属性和“受保护额”属性
- 使用__slots__类属性节省空间
    - 通过__slots__类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而不用字典
    - 继承自超类的__slots__属性没有效果。Python只会使用各个类中定义的__slots__属性
    - 用户定义的类中默认就有__weakref__属性。可是，如果类中定义了__slots__属性，而且想把实例作为弱引用的目标，那么要把'__weakref__'添加到__slots__中。
    - 实例只能拥有__slots__中列出的属性，除非把'__dict__'加入__slots__中（这样做就失去了节省内存的功效）。
- 覆盖类属性
    - 类属性可用于为实例属性提供默认值
    - 如果为不存在的实例属性赋值，会新建实例属性，把同名类属性遮盖
    

In [2]:
# 头文件
from array import array
import math
from datetime import datetime



In [ ]:
# Demo_9-2
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [9]:
# Demo_9-1
v1 = Vector2d(3, 4)
print(v1.x, v1.y)
x, y = v1  # 拆包操作
v1
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
print(v1)
octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1))
print(bool(Vector2d(0,0)))


3.0 4.0
True
(3.0, 4.0)
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
5.0
True
False


In [15]:
# Demo_9-3
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [16]:
octets = b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
v_1 = Vector2d.frombytes(octets)
print(v_1.x, v_1.y)

3.0 4.0


In [17]:
# Demo_9-4

class Demo:
    
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

print(Demo.klassmeth())
print(Demo.klassmeth('one'))

print(Demo.statmeth())
print(Demo.statmeth('one'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'one')
()
('one',)


In [22]:
# 格式说明符
num_a = 100
print(format(num_a, '.3f'))
now = datetime.now()
print(format(now, "%H:%M:%S"))
"it's now {:%I:%M%p}".format(now)

100.000
17:52:31


"it's now 05:52PM"

In [23]:
# Demo_9-5
# 扩展Vectord中的__format__方法
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [25]:
v_2 = Vector2d(3, 4)
print(format(v_2))
print(format(v_2, '.2f'))
print(format(v_2, '.3e'))

(3.0, 4.0)
(3.00, 4.00)
(3.000e+00, 4.000e+00)


In [29]:
# Demo_9-6
# 使用极坐标
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        
        return outer_fmt.format(*components)
        
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [30]:
print(format(Vector2d(1, 1), 'p'))
print(format(Vector2d(1, 1), '.3ep'))
print(format(Vector2d(1, 1), '0.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


In [32]:
v_3 = Vector2d(3, 4)
hash(v_3)
# 对象中的值可以改变，所以不能散列

TypeError: unhashable type: 'Vector2d'

In [2]:
# Demo_9-7
# 让Vector2d的对象变得可以散列

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        
        return outer_fmt.format(*components)
        
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [4]:
v_4 = Vector2d(3, 4)
print(hash(v_4))
print(hash(Vector2d(3.1, 4.2)))

7
384307168202284039


In [13]:
# Demo_9-10
# 名称改写
v_5 = Vector2d(5, 6)
print(v_5.__dict__)
# '_Vector2d__x' 在变量名前增加类名

class ChildVector2d(Vector2d):
    
    def __init__(self, x, y):# 如果要实现名称改写功能，需要自己实现init方法，不能使用父类的
        self.__x = float(x)
        self.__y = float(y)
    
    def __str__(self):
        return super().__str__()
    
cv_1 = ChildVector2d(5, 6)
print(cv_1.__dict__)
print(cv_1) # 这个会导致一个错误，有“名称改写”造成

{'_Vector2d__x': 5.0, '_Vector2d__y': 6.0}
{'_ChildVector2d__x': 5.0, '_ChildVector2d__y': 6.0}


AttributeError: 'ChildVector2d' object has no attribute '_Vector2d__x'

- 批评使用两个下划线这种改写机制的人认为，应该使用命名约定来避免意外覆盖属性
- 他们约定使用一个下划线前缀编写“受保护”的属性（如self._x）

In [ ]:
# Demo_9-11
# __slots__

class Vector2d:
    
    __slots__ = ('__x', '__y') # 告诉解释器：“这个类中的所有实例属性都在这儿了！
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        
        return outer_fmt.format(*components)
        
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)